In [1]:
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt

In [2]:
# change current working directory as kaggle is weird
# import os

# os.chdir('../input/car-damage-detection')
# os.getcwd()

In [3]:
img = cv2.imread('/kaggle/input/car-damage-detection/data1a/training/00-damage/0001.JPEG')
plt.imshow(img)
plt.show()

In [4]:
img.shape

In [5]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=2/255.0,
    featurewise_center=True, 
    samplewise_center=True
)

batch_size = 16

train_datagen = datagen.flow_from_directory(
    directory='/kaggle/input/car-damage-detection/data1a/training',
    target_size=(256, 256),
    batch_size=batch_size,
    subset='training',
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=22
)

validation_generator = datagen.flow_from_directory(
    directory='/kaggle/input/car-damage-detection/data1a/validation',
    target_size=(256, 256),
    batch_size=batch_size,
    subset='training',
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=22
)

In [6]:
w, h, c = (256, 256, 3)

mobilenet = keras.applications.MobileNetV2(
    input_shape=(w,h,c),
    pooling='avg',
    include_top=False,
    weights='imagenet',
    classes=2
)

In [7]:
# mobilenet.summary()

In [8]:
len(mobilenet.layers)

In [9]:
# Total of 175 layers, set first 120 as non-trainable
for layer in mobilenet.layers[:110]:
    layer.trainable = False

for layer in mobilenet.layers[110:]:
    layer.trainable = True
        
mobilenet.summary()

In [10]:
model = keras.models.Sequential([
    mobilenet,
    keras.layers.Dense(512, activation=keras.activations.relu),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(2, activation=keras.activations.softmax)
])

In [11]:
model.summary()

In [13]:
import os
print(os.getcwd())

In [14]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

ckpt_path = os.path.join(os.getcwd(), 'checkpoint')
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path,
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch'
)

earlystopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max'
)

callbacks = [checkpoint, earlystopping]

In [15]:
EPOCHS = 50

history = model.fit(
    train_datagen,
    validation_data=validation_generator,
    steps_per_epoch=train_datagen.n//train_datagen.batch_size,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [16]:
!zip -r file.zip {os.path.join(os.getcwd(), 'checkpoint')}

In [17]:
from IPython.display import FileLink
FileLink('./file.zip')